In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import cv2
from sklearn.manifold import TSNE
from sklearn import svm
from time import time
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [ ]:
from sklearn.decomposition import PCA, KernelPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid

In [ ]:

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
data1 = unpickle("resources/cifar-10-batches-py/data_batch_1")
data2 = unpickle("resources/cifar-10-batches-py/data_batch_2")
data3 = unpickle("resources/cifar-10-batches-py/data_batch_3")
data4 = unpickle("resources/cifar-10-batches-py/data_batch_4")
data5 = unpickle("resources/cifar-10-batches-py/data_batch_5")

In [ ]:
datam = unpickle("resources/cifar-10-batches-py/batches.meta")

In [ ]:
X1 = pd.DataFrame(data1[b'data'])
Y1 = pd.DataFrame(data1[b'labels'])
X2 = pd.DataFrame(data2[b'data'])
Y2 = pd.DataFrame(data2[b'labels'])
X3 = pd.DataFrame(data3[b'data'])
Y3 = pd.DataFrame(data3[b'labels'])
X4 = pd.DataFrame(data4[b'data'])
Y4 = pd.DataFrame(data4[b'labels'])
X5 = pd.DataFrame(data5[b'data'])
Y5 = pd.DataFrame(data5[b'labels'])


In [ ]:
X =pd.concat([X1,X2,X3,X4,X5])

In [ ]:
Y = pd.concat([Y1,Y2,Y3,Y4,Y5])

In [ ]:
X = pd.DataFrame(X)

In [ ]:
#Collecting all the truth label and rename the values with the actuall name of subjects reprecented on images 

Y = np.array(Y)
names=[]
for i in range(Y.shape[0]):
    if Y[i] == 1:
        names.append('airplane')
    elif Y[i]==2:
        names.append("automobile")
    elif Y[i]==3:
        names.append("bird")
    elif Y[i]==4:
        names.append('cat')
    elif Y[i]==3:
        names.append("deer")
    elif Y[i]==5:
        names.append('dog')   
        
    elif Y[i]==6:
        names.append('frog')    
        
    elif Y[i]==7:
        names.append('horse')
    elif Y[i]==7:
        names.append('ship')
    else:
        names.append('truck')
names = pd.DataFrame(names)

In [ ]:
# Reshaping our data.

X = np.array(X)
X = X.reshape(50000, 3, 32, 32).transpose(0,2,3,1).astype("uint8") 

In [ ]:
# Ploting one random image to see how it looks like 

object1 = X[7, :]
object1.shape
plt.imshow(object1, interpolation='nearest')
plt.title("Digit")

In [ ]:
#Performing the tranformatation FROM RGB to GrayScale

XGrey = np.empty_like(X[..., 0])   
for i in range(X.shape[0]):
    XGrey[i] = cv2.cvtColor(X[i], cv2.COLOR_RGB2GRAY)

In [ ]:
#Ploting our result for a random image.

object1 = XGrey[7, :]
object1.shape
plt.imshow(object1,cmap='gray')
plt.title("Digit")

In [ ]:
XGrey.shape

In [ ]:
XGrey = pd.DataFrame(XGrey.reshape(50000,1024))

In [ ]:
XGrey.shape

In [ ]:
XGrey['label']=names.values
cifar10 = pd.DataFrame(XGrey)
cols = list(cifar10.columns)
cols = [cols[-1]] + cols[:-1]
cifar10 = cifar10[cols]

In [ ]:
cifar10

In [ ]:
cifar10 = cifar10[:20000]

In [ ]:
cifar10

In [ ]:
X = cifar10.drop(columns='label')
y = cifar10['label']
X = X/255.0

In [ ]:
X

In [ ]:
#MinMax scaling to our data

from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
Xm = mm.fit_transform(X)


In [ ]:
#Making our problem Binary. By trying to seperate animals from objects.

for i in range(len(y)):
    if (y[i]=="cat"):
        y[i] = 0
    elif (y[i]=='bird'):
        y[i] = 0
    elif (y[i]=='deer'):
        y[i] = 0
    elif(y[i]=='horse'):
        y[i] = 0
    elif (y[i]=='dog'):
        y[i] = 0
    elif(y[i]=='frog'):
        y[i] = 0
    else:
        y[i] = 1


In [ ]:
y = y.astype("int64")

In [ ]:
# Spliting into traing and test set

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xm, y, test_size=0.4, random_state=42)

In [ ]:
y_train

In [ ]:
#Performing PCA to reduce asmuch we can our dimensions but with keeping more than 90% information. 
pca = PCA(0.9)
pca.fit(X_train)
pca.n_components_

In [ ]:
pca = PCA(n_components = 76)
X_trainpca = pca.fit_transform(X_train)
X_testpca = pca.transform(X_test)

In [ ]:
# Now lets plot using t-SNE our data to see how the train test is:
view = TSNE(n_components=2, random_state=123).fit_transform(X_trainpca)
plt.figure(figsize=(20,10))
plt.scatter(view[:,0], view[:,1], c=y_train, alpha=0.5)
plt.xlabel('t-SNE-1')
plt.ylabel('t-SNE-2')

In [ ]:
# Defining Hyperparametrs for our model:
paramC = [0.001, 0.01, 0.1, 1, 5]
gamm = ['auto','scale']
deg = [3, 6, 8]


In [ ]:
# Start building our svm with linear kernel:
Cl=[]
Al=[]
Atl=[]
for i in paramC:
    start = time()
    clf = svm.SVC(kernel = 'linear', C=i)
    clf.fit(X_trainpca, y_train)
    accu = round(accuracy_score(y_train, clf.predict(X_trainpca)),4)
    acct = round( accuracy_score(y_test, clf.predict(X_testpca)),4)
    end = time()
    Cl.append(i)
    Al.append(accu*100)
    Atl.append(acct*100)
    print("Done in=", (end-start)//1,"   Accuracy score=",accu*100,"Αccuracy Test",acct*100,"for C=",i,acct*100 )

In [ ]:
# Results for Linear Kernel
tl=[1.07,1.1,1.2,1.9,4.33]
resultslin = pd.DataFrame({'time(s) ' : tl,
                         'C' : Cl,
                        "Accuracy Score Train" : Al,
                          "Accuracy Score Test" : Atl},
                        columns=['time(s) ','C',"Accuracy Score Train","Accuracy Score Test"])

In [ ]:
resultslin.sort_values("Accuracy Score Test",ascending=True)

In [ ]:
# Resulting plots for linear kernel 
plt.figure(figsize=(8,6))
plt.plot(resultslin["C"], resultslin["Accuracy Score Train"])
plt.plot(resultslin["C"], resultslin["Accuracy Score Test"])
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.title("Kernel Linear")
plt.ylim(60,100)
plt.legend(['Accuracy Train','Accuracy Test'])
plt.xscale("log")

In [ ]:
# Start building our svm with polynomial kernel:
Cp=[]
Gp=[]
Dp=[]
Ap=[]
Apt=[]
for i in paramC:
    for k in gamm:
        for j in deg:
            start = time()
            clf = svm.SVC(kernel = 'poly', C=i, gamma=k,degree=j)
            clf.fit(X_trainpca, y_train)
            acc = round( accuracy_score(y_train, clf.predict(X_trainpca)),4)
            acct = round( accuracy_score(y_test, clf.predict(X_testpca)),4)
            end = time()
            Cp.append(i)
            Gp.append(k)
            Dp.append(j)
            Ap.append((acc*100))
            Apt.append(acct*100)
            print("Done in=", (end-start)//1,"Accuracy score=",acc*100,"Accuracy Test",acct*100,"for C=",i,"for gamma=",k," for degree=",j)


In [ ]:
# Results for Polynomial Kernel
tpo=[1.42,1.43,1.48,1.4,1.42,1.45,1.42,1.47,1.3,1.38,1.43,1.23,1.38,1.43,1.23,1.38,1.48,1.23,1.33,1.4,1.23,1.33,1.42,1.32,1.32,1.4,1.4,2.15,2.17,2.05]
resultspoly = pd.DataFrame({'time(s) ':tpo,
                           'C' : Cp,
                           'Gamma' : Gp,
                           "Degree" :Dp,
                           "Accuracy Score Train" : Ap,
                           "Accuracy Score Test":Apt},
                        columns=['time(s) ','C','Gamma',"Degree","Accuracy Score Train","Accuracy Score Test"])

In [ ]:
resultspoly.sort_values("Accuracy Score Test",ascending=True)

In [ ]:
# Resulting plots for Polynomial Kernel 
plt.figure(figsize=(19,6))

plt.subplot(131)
D1 = resultspoly[resultspoly['Degree']==3]
plt.plot(D1["C"], D1["Accuracy Score Train"])
plt.plot(D1["C"], D1["Accuracy Score Test"])
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.title("Degree = 3")
plt.ylim(10,100)
plt.legend(['Accuracy Train','Accuracy Test'])
plt.xscale("log")

plt.subplot(132)
D2 = resultspoly[resultspoly['Degree']==6]
plt.plot(D2["C"], D2["Accuracy Score Train"])
plt.plot(D2["C"], D2["Accuracy Score Test"])
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.title("Degree = 6")
plt.ylim(10,100)
plt.legend(['Accuracy Train','Accuracy Test'])
plt.xscale("log")

plt.subplot(133)
D3 = resultspoly[resultspoly['Degree']==8]
plt.plot(D3["C"], D3["Accuracy Score Train"])
plt.plot(D3["C"], D3["Accuracy Score Test"])
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.title("Degree = 8")
plt.ylim(10,100)
plt.legend(['Accuracy Train','Accuracy Test'])
plt.xscale("log")


In [ ]:
# Start building our svm with RBF  kernel:

Cr=[]
Gr=[]
Ar=[]
Atr=[]
for i in paramC:
    for k in gamm:
        start1 = time()
        clf = svm.SVC(kernel = 'rbf', C=i, gamma=k)
        clf.fit(X_trainpca, y_train)
        accu = round(accuracy_score(y_train, clf.predict(X_trainpca)),4)
        acct = round( accuracy_score(y_test, clf.predict(X_testpca)),4)
        end1 = time()
        Cr.append(i)
        Gr.append(k)
        Ar.append((accu*100))
        Atr.append(acct*100)        
        print("Done in=", (end1-start1)//1,"Accuracy score Train=",accu*100,"Accuracy Score Test",acct*100,"for C=",i,"for gamma=",k)

In [ ]:
#Results for RBF Kernel

tr=[1.57,1.57,1.35,1.38,1.18,1.25,1.05,1.08,1.17,1.32]
resultsrbf = pd.DataFrame({'time(s) ' : tr,
                         'C' : Cr,
                        'Gamma' : Gr ,
                       "Accuracy Score Train" : Ar,
                          "Accuracy Score Test":Atr},
                        columns=['time(s) ','C', 'Gamma',"Accuracy Score Train","Accuracy Score Test"])

In [ ]:
resultsrbf.sort_values("Accuracy Score Test",ascending=True)

In [ ]:
# Resulting plots for RBF kernel 
plt.figure(figsize=(12,6))
plt.subplot(121)
gam1 = resultsrbf[resultsrbf['Gamma']=="auto"]
plt.plot(gam1["C"], gam1["Accuracy Score Train"])
plt.plot(gam1["C"], gam1["Accuracy Score Test"])
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.title("Auto")
plt.ylim(10,100)
plt.legend(['Accuracy Train','Accuracy Test'])
plt.xscale("log")

plt.subplot(122)
gam2 = resultsrbf[resultsrbf['Gamma']=="scale"]
plt.plot(gam2["C"], gam2["Accuracy Score Train"])
plt.plot(gam2["C"], gam2["Accuracy Score Test"])
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.title("Scale")
plt.ylim(10,100)
plt.legend(['Accuracy Train','Accuracy Test'])
plt.xscale("log")


In [ ]:
#Results for the Best Accuracy Scores

Kernel = ["Linear", "Polynomial","RBF"]
timeer = ["4.33", "1.38","1.25"]
C = [5,0.01,0.1]
G = ["-","scale","scale"]
D = ["-",3,"-"]
As =[67.35,64.32,72.29]
results = pd.DataFrame({'Kernel':Kernel,
                          'Time':timeer,
                        'C' : C,
                        'Gamma' : G ,
                        'Degree' : D,
                         "Accuracy Score Train":As},
                        columns=['Kernel','Time','C', 'Gamma','Degree',"Accuracy Score Train"])

In [ ]:
results

In [ ]:
# Now we will see the results for our best kernel for both train and test set. 
clf = svm.SVC(kernel="rbf", C=0.1,gamma="scale")
clf.fit(X_trainpca, y_train)
#Predictions
y_train_pred = clf.predict(X_trainpca)
y_test_pred = clf.predict(X_testpca)
#Accuracy score
print('Accuracy Score on Train Set: {:.2f}%'.format(accuracy_score(y_train,y_train_pred)*100))
print('Accuracy Score on Test Set: {:.2f}%'.format(accuracy_score(y_test,y_test_pred)*100))

In [ ]:
# Confusion matrix for X_train

CMTrain = pd.DataFrame(confusion_matrix(y_true=y_train,y_pred=y_train_pred),
                  columns=pd.MultiIndex.from_product([["Prediction"],['Negative',"Positive"]]),
                  index=pd.MultiIndex.from_product([["Actual"],["Negative","Positive"]]))

In [ ]:
CMTrain

In [ ]:
# Confusion matrix for X_test
CMTest = pd.DataFrame(confusion_matrix(y_test,y_test_pred),
                  columns=pd.MultiIndex.from_product([["Prediction"],['Negative',"Positive"]]),
                  index=pd.MultiIndex.from_product([["Actual"],["Negative","Positive"]]))

In [ ]:
CMTest

In [ ]:
print(classification_report(y_test,y_test_pred))

In [ ]:

#Cross validation for the k-NN classifier
X_traind,X_val,y_traind,y_val = train_test_split(X_trainpca,y_train,test_size=0.1,random_state=84)

In [ ]:
#k-NN classification 

kn = np.arange(1,30,2)
AcValid=[]
start=time()
for k in kn:
    
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_traind,y_traind)
    score = round(knn.score(X_val, y_val),4)
    AcValid.append(score*100)
    end = time()
    print("Done in:",(end-start)//1,"k=%d,  accuracy=%.2f%%" % (k, score * 100))
                          

In [ ]:
Tr = [0.03,0.07,0.11,0.15,0.19,0.23,0.28,0.32,0.36,0.40,0.45,0.49,0.53,0.58,1.02]
resultsknn = pd.DataFrame({'time(s) ' : Tr,
                        'n_Neighbors' : kn ,
                       "Accuracy score" : AcValid},
                        columns=['time(s) ', 'n_Neighbors',"Accuracy score"])

In [ ]:
resultsknn.sort_values("Accuracy score",ascending=True)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=11)
knn.fit(X_trainpca,y_train)
y_testpred = knn.predict(X_testpca)
print('Accuracy Score on Train Set: {:.2f}%'.format(accuracy_score(y_train,knn.predict(X_trainpca))*100))
print('Accuracy Score on Test Set: {:.2f}%'.format(accuracy_score(y_test,knn.predict(X_testpca))*100))
print(classification_report(y_test,knn.predict(X_testpca))

In [ ]:
nc = NearestCentroid()
nc.fit(X_trainpca,y_train)
print('Accuracy Score on Train Set: {:.2f}%'.format(accuracy_score(y_train,nc.predict(X_trainpca))*100))
print('Accuracy Score on Test Set: {:.2f}%'.format(accuracy_score(y_test,nc.predict(X_testpca))*100))
print(classification_report(y_test,nc.predict(X_testpca)))

In [ ]:
# KPCA+ LDA method using k-NN classifier.

co = np.arange(1,100,10)
Acc=[]
Acct=[]
for i in co:
    
    # KPCA with rbf kernel 
    start = time()
    kp = KernelPCA(n_components=i,kernel='rbf')
    X_trainkpca = kp.fit_transform(X_train)
    X_testkpca = kp.transform(X_test)
    
    # LDA using n_components as 
    ld = LinearDiscriminantAnalysis(n_components=1)
    X_trainl = ld.fit_transform(X_trainkpca,y_train)
    X_testl = ld.transform(X_testkpca)
    
    # fiting our model using k-NN classifier 
    knn = KNeighborsClassifier(n_neighbors=17)
    knn.fit(X_trainl,y_train)
    #Calculating accuracy
    accTr = round(accuracy_score(y_train,knn.predict(X_trainl)),4)*100
    accT = round(accuracy_score(y_test,knn.predict(X_testl)),4)*100
    Acc.append(accTr)
    Acct.append(accT)
    end = time()
    print("Done in: ",(end-start)//1,"Accuracy train : ",accTr,"Accuracy test : ",accT)
    

In [ ]:
#Results for KPCA-LDA for k-NN classification 

tkp=[0.588.13,9.57,10.30,10.38,10.10,10.20,10,10.33,10.56]
resultskn = pd.DataFrame({'time(s) ' : tkp,
                           "components":co,
                       "Accuracy Score Train" : Acc,
                          "Accuracy Score Test":Acct},
                        columns=['time(s) ','components',"Accuracy Score Train","Accuracy Score Test"])
resultskn.sort_values("Accuracy Score Test",ascending=True)

In [ ]:
# KPCA+ LDA method using Nearest Centroid.

co = np.arange(1,100,10)
Anc=[]
Anct=[]
for i in co:
    start = time()
    # KPCA with rbf kernel 
    kp = KernelPCA(n_components=i,kernel='rbf')
    X_trainkpca = kp.fit_transform(X_train)
    X_testkpca = kp.transform(X_test)
    
    # LDA using n_components as 
    ld = LinearDiscriminantAnalysis(n_components=1)
    X_trainl = ld.fit_transform(X_trainkpca,y_train)
    X_testl = ld.transform(X_testkpca)
    
    # fiting our model using Nearest Centriod classifier 
    nc = NearestCentroid()
    nc.fit(X_trainl,y_train)
    #calculating accuracy 
    accTr = round(accuracy_score(y_train,nc.predict(X_trainl)),4)*100
    accT = round(accuracy_score(y_test,nc.predict(X_testl)),4)*100
    Anc.append(accTr)
    Anct.append(accT)
    end = time()
    print("Done in: ",(end-start)//1,"Accuracy train : ",accTr,"Accuracy test : ",accT)

In [ ]:
#Results for KPCA-LDA for Nearest centroid classification 

trn=[1.07,10.30,9.25,10.07,10.45,10.25,10.31,10,10.37]
resultsnc = pd.DataFrame({'time(s) ' : trn,
                           "components":co,
                       "Accuracy Score Train" : Anc,
                          "Accuracy Score Test":Anct},
                        columns=['time(s) ',"components","Accuracy Score Train","Accuracy Score Test"])
resultsnc.sort_values("Accuracy Score Test",ascending=True)

In [ ]:
# Resulting plots

plt.figure(figsize=(10,6))
#KPCA+LDA ploting for accuracy scores using k-NN classifier
plt.subplot(1,2,1)
plt.plot(resultskn["components"], resultskn["Accuracy Score Train"])
plt.plot(resultskn["components"], resultskn["Accuracy Score Test"])
plt.xlabel('components')
plt.ylabel('Accuracy')
plt.title("KPCA+LDA k-NN")
plt.ylim(40,100)
plt.plot()
plt.legend(['Accuracy Train','Accuracy Test'])

#KPCA+LDA ploting for accuracy scores using Nearest Centroid classifier
plt.subplot(1,2,2)
plt.plot(resultsnc["components"], resultsnc["Accuracy Score Train"])
plt.plot(resultsnc["components"], resultsnc["Accuracy Score Test"])
plt.xlabel('components')
plt.ylabel('Accuracy')
plt.title("KPCA+LDA Nearest Centroid")
plt.ylim(40,100)
plt.legend(['Accuracy Train','Accuracy Test'])

In [ ]:
    # KPCA + LDA k-NN classifier 
kp = KernelPCA(n_components=31,kernel='rbf')
X_trainkpca = kp.fit_transform(X_train)
X_testkpca = kp.transform(X_test)
    

ld = LinearDiscriminantAnalysis(n_components=1)
X_trainl = ld.fit_transform(X_trainkpca,y_train)
X_testl = ld.transform(X_testkpca)
    
 
knn = KNeighborsClassifier(n_neighbors=11)
knn.fit(X_trainl,y_train)
    #Calculating accuracy
print('Accuracy Score on Train Set: {:.2f}%'.format(accuracy_score(y_train,knn.predict(X_trainl))*100))
print('Accuracy Score on Test Set: {:.2f}%'.format(accuracy_score(y_test,knn.predict(X_testl))*100))
print(classification_report(y_test,knn.predict(X_testl)))

In [ ]:
    # KPCA + LDA Nearest Centroid 
kp = KernelPCA(n_components=81,kernel='rbf')
X_trainkpca = kp.fit_transform(X_train)
X_testkpca = kp.transform(X_test)
    
ld = LinearDiscriminantAnalysis(n_components=1)
X_trainl = ld.fit_transform(X_trainkpca,y_train)
X_testl = ld.transform(X_testkpca)
    

nc = NearestCentroid()
nc.fit(X_trainl,y_train)
print('Accuracy Score on Train Set: {:.2f}%'.format(accuracy_score(y_train,nc.predict(X_trainl))*100))
print('Accuracy Score on Test Set: {:.2f}%'.format(accuracy_score(y_test,nc.predict(X_testl))*100))
print(classification_report(y_test,nc.predict(X_testl)))

In [ ]:
#The final results for all the methods. 

Method = ["SVM", "k-NN","Nearest Centroid","KPCA+LDA k-NN","KPCA+LDA Nearest Centroid"]
timeer = [1.25,0.23,0.18,9.47,8.02]
At = [72.29,73.55,64.28,70.13,68.46]
Atest = [70.86,67.81,64.21,64.64,67.67]
resultsfinal = pd.DataFrame({'Method':Method,
                          'Time(s) ':timeer,
                        'Accuracy Train Set' : At,
                         "Accuracy Test Set":Atest},
                        columns=['Method','Time(s) ','Accuracy Train Set',"Accuracy Test Set"])

In [ ]:
resultsfinal